### Import

In [0]:
#!pip install sklearn_pandas

**Without stop split and stacking**

~~One entry of Trujet - replace with SpiceJet~~  
JA = 80,000 , JAB = 50,000 & SJ = 25000(5136), looks like outliers  
~~After weekday add month as well~~  
~~Should we mark all above 50000 from JA as JAB?~~  
~~Stratefy based on price bins, Airline, Months~~  
End date on Sat or Sun  
~~Duration is long or short~~ bad may be tune durationTime 

KNN, SVR, Ada,light,gbm  
Excess time in flight.


In [0]:
def temp_to_min(X):
    hour, minute = pd.Series(X).str.split(" ", 1).str
    hour = pd.Series(hour).str.replace("h", " ")
    hour = hour.astype('int64')
    minute = pd.Series(minute).str.replace("m", " ")
    minute = minute.fillna(0)
    minute = minute.astype('int64')
    total_minute = pd.to_timedelta( (hour * 60) + minute, unit = "m" )
    return(total_minute)

### Load modules

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn_pandas import DataFrameMapper, FunctionTransformer, CategoricalImputer
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Data Cleaning

In [0]:
train_data = pd.read_csv("drive/My Drive/PyColab Work/FlightPricePredict/train.csv")
sub_data = pd.read_csv("drive/My Drive/PyColab Work/FlightPricePredict/test.csv")
#train_data.drop(index = 6474, inplace = True, axis = 0) # 5m flight time #change to 24hr 5m
train_data.iloc[6474, 7] = "24h 5m"
sub_data.iloc[2660, 7] = "24h 5m"

train_data.iloc[1478, 0] = "Jet Airways Business"
train_data.iloc[2618, 0] = "Jet Airways Business"
train_data.iloc[5439, 0] = "Jet Airways Business"

train_data.drop(index = 2924, inplace = True, axis = 0)
#train_data.drop(index = 5136, inplace = True, axis = 0)
#train_data.drop(index = 2618, inplace = True, axis = 0)
#train_data.drop(index = 5439, inplace = True, axis = 0)




# Date need to be looked at # Move this
train_data["Week"] = pd.to_datetime(train_data["Date_of_Journey"], dayfirst = True).dt.weekday_name
train_data["Month"] = pd.to_datetime(train_data["Date_of_Journey"], dayfirst = True).dt.month
train_data["Airline"] = train_data.Airline.replace("Trujet", "SpiceJet")
train_data["class"] = np.where(train_data['Airline']=='Jet Airways Business', 'High', 'Low')
train_data["meal"] = np.where(train_data['Additional_Info']=='In-flight meal not included', 'High', 'Low')
train_data["checkin"] = np.where(train_data['Additional_Info']=='No check-in baggage included', 'High', 'Low')
train_data["Destination"] = np.where(train_data["Destination"] == "New Delhi", "Delhi", train_data["Destination"])
train_data["Airline"] = np.where(train_data["Airline"] == "Vistara Premium economy", "Multiple carriers Premium economy", train_data["Airline"])
train_data['landing'] = pd.to_datetime(train_data['Date_of_Journey'] + '-' + train_data['Dep_Time'], dayfirst = True) + temp_to_min(train_data["Duration"])
train_data["landingWeek"] = pd.to_datetime(train_data["landing"]).dt.weekday_name
train_data["Source"] = train_data["Source"] + train_data["Destination"]
train_data.head()

#Seperate y here itself

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Week,Month,class,meal,checkin,landing,landingWeek
0,IndiGo,24/03/2019,BangloreDelhi,Delhi,BLR ? DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,Sunday,3,Low,Low,Low,2019-03-25 01:10:00,Monday
1,Air India,1/05/2019,KolkataBanglore,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662,Wednesday,5,Low,Low,Low,2019-05-01 13:15:00,Wednesday
2,Jet Airways,9/06/2019,DelhiCochin,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,Sunday,6,Low,Low,Low,2019-06-10 04:25:00,Monday
3,IndiGo,12/05/2019,KolkataBanglore,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218,Sunday,5,Low,Low,Low,2019-05-12 23:30:00,Sunday
4,IndiGo,01/03/2019,BangloreDelhi,Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302,Friday,3,Low,Low,Low,2019-03-01 21:35:00,Friday


In [0]:
%%script false

def flight_data_clean(df) :
  df["Week"] = pd.to_datetime(df["Date_of_Journey"]).dt.weekday
  df["Month"] = pd.to_datetime(df["Date_of_Journey"]).dt.month
  df = df.drop(['Date_of_Journey','Route', 'Additional_Info'], axis = 1)
  df = pd.get_dummies(df, columns=['Airline', 'Source', 'Destination', 'Total_Stops'])
  return(df)

flight_data_clean(train_data)

In [0]:
sub_data["Week"] = pd.to_datetime(sub_data["Date_of_Journey"], dayfirst = True).dt.weekday_name
sub_data["Month"] = pd.to_datetime(sub_data["Date_of_Journey"], dayfirst = True).dt.month
sub_data["class"] = np.where(sub_data['Airline']=='Jet Airways Business', 'High', 'Low')
sub_data["meal"] = np.where(sub_data['Additional_Info']=='In-flight meal not included', 'High', 'Low')
sub_data["checkin"] = np.where(sub_data['Additional_Info']=='No check-in baggage included', 'High', 'Low')
sub_data["Destination"] = np.where(sub_data["Destination"] == "New Delhi", "Delhi", sub_data["Destination"])
sub_data["Airline"] = np.where(sub_data["Airline"] == "Vistara Premium economy", "Multiple carriers Premium economy", sub_data["Airline"])
sub_data['landing'] = pd.to_datetime(sub_data['Date_of_Journey'] + '-' + sub_data['Dep_Time'], dayfirst = True) + temp_to_min(sub_data["Duration"])
sub_data["landingWeek"] = pd.to_datetime(sub_data["landing"]).dt.weekday_name
sub_data["Source"] = sub_data["Source"] + sub_data["Destination"]

In [0]:
encoder = LabelBinarizer() # LabelBinarizer()  ~~#LabelEncoder()~~
to_bin = KBinsDiscretizer(n_bins=3, encode = 'onehot-dense')


class to_min(TransformerMixin):
  
  def fit(self, X, y=None):
    return self
  
  def transform(self, X):
    hour, minute = pd.Series(X).str.split(" ", 1).str
    hour = pd.Series(hour).str.replace("h", " ")
    hour = hour.astype('int64')
    minute = pd.Series(minute).str.replace("m", " ")
    minute = minute.fillna(0)
    minute = minute.astype('int64')
    total_minute = pd.Series( (hour * 60) + minute )
    #short_dist = pd.Series( np.where(total_minute < 360, 1, 0) )
    #long_dist = pd.Series( np.where(total_minute < 360, 0, 1) )
    #return (pd.concat([total_minute, short_dist, long_dist], axis = 1))
    total_minute = pd.DataFrame(to_bin.fit_transform(pd.DataFrame(total_minute)))
    return(total_minute)

# Can we use subclass?
class part_of_day(TransformerMixin):
  def fit(self, X, y = None):
    return self
  
  def transform(self, X):
    col = pd.Series(X) 
    aseries = col.replace(":\d\d.*", "", regex = True).astype('int32')
    conditions = [
      (aseries > 6) & (aseries <= 13),
      (aseries > 13) & (aseries <= 18),
      (aseries > 18) & (aseries <= 21),
      (aseries > 21) | (aseries <= 6)
    ]

    choices = ['morning', 'afternoon', 'eve', 'night']
  
    return np.select(conditions, choices, default='night')
  

class convert_stops(TransformerMixin):
  def fit(self, X, y = None):
    return self
  
  def transform(self, X):
    col = pd.Series(X) 
    #aseries = 
    conditions = [
      (col == "non-stop"),
      (col == "1 stop"),
      (col == "2 stops"),
      (col == "3 stops"),
      (col == "4 stops")
    ]

    choices = [0, 1, 2, 3, 4]
  
    return np.select(conditions, choices, default='0')

In [0]:

# As Route and Additional_Info are not defined they are dropped. 
# Write a default function for imputation

mapper = DataFrameMapper([
    ( 'Airline', encoder ),
    ( 'Week', encoder ),
    ( 'Month', encoder ),
    ( 'Source', encoder ),
    #( 'Destination', encoder ),
    ( 'Dep_Time', [part_of_day(), encoder] ),
    ( 'Arrival_Time', [part_of_day(), encoder] ),
    ( 'Duration', to_min() ),
    ( 'Total_Stops', [CategoricalImputer(strategy = "most_frequent"), convert_stops()] ),
    ( 'class', encoder),
    ( 'meal', encoder),
    ( 'landingWeek', encoder),
    ( 'checkin', encoder) # As these columns are binary, encoder does not apply on them.
    #( 'Price', FunctionTransformer(np.log) ),
], df_out = True
)

In [0]:
X = train_data.drop("Price", axis = 1)
y = np.log(train_data['Price']).astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, stratify = X['Airline']) #Stratify

#X_test, X_ftest, y_test, y_ftest = train_test_split(X_test, y_test, test_size = 0.10, stratify = X_test['Airline'])

In [0]:
# #Prepare pipeline and fit there.
X_train = mapper.fit_transform(X_train).astype(float)#.drop(['Airline_Multiple carriers Premium economy'], axis = 1)
X_test = mapper.fit_transform(X_test).astype(float)#.drop(['Airline_Multiple carriers Premium economy'], axis = 1)

#Can we drop 'Airline_Multiple carriers Premium economy' in onehotencoder and use stacking?
#X_ftest = mapper.fit_transform(X_ftest).astype(float)#.drop(['Airline_Multiple carriers Premium economy'], axis = 1)
sub = mapper.fit_transform(sub_data).astype(float)#.drop(['Airline_Multiple carriers Premium economy'], axis = 1)

In [0]:
X_train.columns

Index(['Airline_Air Asia', 'Airline_Air India', 'Airline_GoAir',
       'Airline_IndiGo', 'Airline_Jet Airways', 'Airline_Jet Airways Business',
       'Airline_Multiple carriers',
       'Airline_Multiple carriers Premium economy', 'Airline_SpiceJet',
       'Airline_Vistara', 'Week_Friday', 'Week_Monday', 'Week_Saturday',
       'Week_Sunday', 'Week_Thursday', 'Week_Tuesday', 'Week_Wednesday',
       'Month_3', 'Month_4', 'Month_5', 'Month_6', 'Source_BangloreDelhi',
       'Source_ChennaiKolkata', 'Source_DelhiCochin', 'Source_KolkataBanglore',
       'Source_MumbaiHyderabad', 'Dep_Time_afternoon', 'Dep_Time_eve',
       'Dep_Time_morning', 'Dep_Time_night', 'Arrival_Time_afternoon',
       'Arrival_Time_eve', 'Arrival_Time_morning', 'Arrival_Time_night',
       'Duration_0', 'Duration_1', 'Duration_2', 'Total_Stops', 'class',
       'meal', 'landingWeek_Friday', 'landingWeek_Monday',
       'landingWeek_Saturday', 'landingWeek_Sunday', 'landingWeek_Thursday',
       'landingWeek_Tu

## Boosting

Boosting in following format  
Model based on default values  
Model result on 5 parts of train data, using cv  
RMSE on test data

Hyperparameter tuning with GridSearch cv  
grid result on 5 parts of train data  
RMSE of test data using grid  

Define boosting function

### XGBoost

In [0]:
base_model = xgb.XGBRegressor(learning_rate=0.2, gamma= 0.01, max_depth = 7)
base_model.fit(X_train, y_train)
print( np.sqrt( -cross_val_score(base_model, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
np.sqrt(mean_squared_error(np.exp(y_test), np.exp(base_model.predict(X_test))))

[0.178591   0.17616793 0.17390439 0.17912586 0.18019413]


1791.5764380778703

In [0]:
xgb_engine = xgb.XGBRegressor() #n_estimator not used
xgb_params = {'max_depth' : np.arange(1, 10, 3), 'gamma' : [0.01, 1], 'learning_rate' : [0.1, 0.2, 0.3]}

xgb_grid = GridSearchCV(xgb_engine, xgb_params, cv = 5, n_jobs = -1, verbose = 1)
xgb_grid.fit(X_train, y_train)

print( np.sqrt( -cross_val_score(xgb_grid.best_estimator_, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
print( np.sqrt( mean_squared_error(np.exp(y_test), np.exp(xgb_grid.predict(X_test))) ) )

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   37.0s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.2min finished


[0.178591   0.17616793 0.17390439 0.17912586 0.18019413]
1791.5764380778703


### GBM

In [0]:
gbm_model = GradientBoostingRegressor(learning_rate = 0.05, max_depth = 7, n_estimators = 300)
gbm_model.fit(X_train, y_train)
print( np.sqrt( -cross_val_score(gbm_model, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
np.sqrt(mean_squared_error(np.exp(y_test), np.exp(gbm_model.predict(X_test))))

[0.17969775 0.17595766 0.17173887 0.18166729 0.17842296]


1807.067138106383

In [0]:
gbm_engine = GradientBoostingRegressor()
gbm_params = {'max_depth' : [7], 'learning_rate' : [0.05], 'n_estimators' : [300]}

gbm_grid = GridSearchCV(gbm_engine, gbm_params, cv = 5, n_jobs = -1, verbose = 1)
gbm_grid.fit(X_train, y_train)
print( np.sqrt( -cross_val_score(gbm_grid.best_estimator_, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
print( np.sqrt( mean_squared_error(np.exp(y_test), np.exp(gbm_grid.predict(X_test))) ) )

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   42.0s finished


[0.17870984 0.17615369 0.17154961 0.18142573 0.17863615]
1816.3764133003674


### Light GBM

In [0]:
# https://lightgbm.readthedocs.io/en/latest/Python-API.html#scikit-learn-api
# Use gridsearchcv

import lightgbm as lgb
lgbm_model = lgb.LGBMRegressor(learning_rate = 0.3, n_estimators = 300, max_depth = 6, min_child_samples  = 17)
lgbm_model.fit(X_train.astype(float), y_train.astype(float))

print( np.sqrt( -cross_val_score(lgbm_model, X_train.astype(float), y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
np.sqrt(mean_squared_error(np.exp(y_test), np.exp(lgbm_model.predict(X_test))))

[0.18349371 0.18203957 0.17739656 0.18536029 0.18237333]


2032.2712991110502

In [0]:
lgbm_engine = lgb.LGBMRegressor()
lgbm_params = {'max_depth' : [6], 'learning_rate' : [ 0.3], 'n_estimators': [300], 'reg_alpha' : [0],  'min_child_samples' : [17]}

lgbm_grid = GridSearchCV(lgbm_engine, lgbm_params, cv = 5, n_jobs = -1, verbose = 0)
lgbm_grid.fit(X_train, y_train)
print( np.sqrt( -cross_val_score(lgbm_grid.best_estimator_, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
print( np.sqrt( mean_squared_error(np.exp(y_test), np.exp(lgbm_grid.predict(X_test))) ) )

[0.18349371 0.18203957 0.17739656 0.18536029 0.18237333]
2032.2712991110502


In [0]:
for a, b, c in zip( lgbm_grid.cv_results_['params'], lgbm_grid.cv_results_['mean_test_score'], lgbm_grid.cv_results_['mean_train_score'] ):
  print(a, b, c)

{'learning_rate': 0.3, 'max_depth': 6, 'min_child_samples': 17, 'n_estimators': 300, 'reg_alpha': 0} 0.8734111632010509 0.9294670517830271


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


### AdaBoost

In [0]:
from sklearn.ensemble import AdaBoostRegressor

adb_model = AdaBoostRegressor(n_estimators = 300, learning_rate = 0.2)
adb_model.fit(X_train, y_train)

print( np.sqrt( -cross_val_score(adb_model, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
np.sqrt(mean_squared_error(np.exp(y_test), np.exp(adb_model.predict(X_test))))

[0.29303053 0.29382685 0.28953325 0.29818444 0.30160162]


3073.3280972438133

In [0]:
adb_engine = AdaBoostRegressor()
adb_params = {'n_estimators' : np.arange(50, 500, 50), 'learning_rate' : [0.1, 0.2, 0.3]}

adb_grid = GridSearchCV(adb_engine, adb_params, cv = 5, n_jobs = -1, verbose = 1)
adb_grid.fit(X_train, y_train)
print( np.sqrt( -cross_val_score(adb_grid.best_estimator_, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
print( np.sqrt( mean_squared_error(np.exp(y_test), np.exp(adb_grid.predict(X_test))) ) )

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   59.3s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:  2.0min finished


[0.28912188 0.28287447 0.28223887 0.30034243 0.2959948 ]
2957.2918846920384


## Bagging

### Random Forest

In [0]:
from sklearn.ensemble import RandomForestRegressor

ranger_model = RandomForestRegressor(n_estimators = 100, oob_score = True)
ranger_model.fit(X_train, y_train)
print( np.sqrt( -cross_val_score(ranger_model, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
np.sqrt(mean_squared_error(np.exp(y_test), np.exp(ranger_model.predict(X_test))))

[0.18422773 0.18397932 0.18712185 0.19166247 0.18764033]


1889.2635548759438

In [0]:
ranger_engine = RandomForestRegressor()
ranger_params = {'n_estimators' : np.arange(50, 500, 50)}

ranger_grid = GridSearchCV(ranger_engine, ranger_params, cv = 5, n_jobs = -1, verbose = 1)
ranger_grid.fit(X_train, y_train)
print( np.sqrt( -cross_val_score(ranger_grid.best_estimator_, X_train, y_train, cv = 5, scoring = "neg_mean_squared_error") ) )
print( np.sqrt( mean_squared_error(np.exp(y_test), np.exp(ranger_grid.predict(X_test))) ) )

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  3.3min finished


[0.18443997 0.18406537 0.18541716 0.19004336 0.18702247]
1876.679538470102


## Ensemble

### Average ensemble

In [0]:
# Adaboost overfitting
outcome = pd.DataFrame( {'-actual' : np.exp(y_test) , 
               'a-xgb' :np.exp(xgb_grid.predict(X_test)),
               'b-gbm' : np.exp(gbm_grid.predict(X_test)),
               'c-lgbm' : np.exp(lgbm_grid.predict(X_test))
                        } )
# 'd-ranger' : np.exp(ranger_grid.predict(X_test)) 
outcome['avg'] = outcome.mean(axis = 1)
outcome.describe()

,-actual,a-xgb,b-gbm,c-lgbm,avg
count,2137.000000,2137.000000,2137.000000,2137.000000,2137.000000
mean,8950.171736,8949.885742,8942.968913,8916.697769,8939.933435
std,4520.024280,4251.795898,4250.441276,4039.134974,4169.225012
min,1759.000000,1924.979736,1928.384833,1890.183717,1933.932452
25%,5135.000000,5482.491211,5419.284677,5396.261804,5429.892900
50%,8073.000000,8821.814453,8773.178055,8691.787748,8775.474397
75%,12373.000000,11545.903320,11507.910968,11524.779215,11692.389376
max,57209.000000,61692.621094,62134.821177,31113.811850,51067.084038


In [0]:
# ens_predict = pd.DataFrame( { 'axgbpre' :np.exp(base_model.predict(X_test)) ,
#                'brfpre' : np.exp(ranger.predict(X_test)) } ).mean(axis = 1)

ens_predict = outcome['avg']

np.sqrt( mean_squared_error(np.exp(y_test), ens_predict) )

1349.110237805378

Submit file

In [0]:
#%%script false

soutcome = pd.DataFrame({'a-xgb' :np.exp(xgb_grid.predict(sub)),
               'b-gbm' : np.exp(gbm_grid.predict(sub)),
               'c-lgbm' : np.exp(lgbm_grid.predict(sub.astype(float))) } )

soutcome['avg'] = soutcome.mean(axis = 1)

pd.DataFrame({ "Price" : soutcome['avg'] }, index = None).to_csv("FlightPriceEn.csv")

### Stacking

A pipeline needed

In [0]:
%%script false

X_stack = pd.DataFrame( {'axgbpre' :base_model.predict(X_test) ,
                         'gbm' : gbm.predict(X_test),
                         'lgbm' : lgbm.predict(X_test),
                          'adb' : adb.predict(X_test)})

y_stack = y_test

In [0]:
%%script false

stack_model = xgb.XGBRegressor(learning_rate=0.05, n_estimators = 1000) #learning_rate=0.2, gamma= 0.01, max_depth = 7
#stack_model.fit(X_stack, y_stack)
params = {"max_depth" : [2, 5, 7]}
ensemble_grid = GridSearchCV(stack_model, params, cv = 5)
ensemble_grid.fit(X_stack, y_stack)

In [0]:
%%script false

ensemble_grid.best_estimator_

In [0]:
%%script false

np.sqrt( -cross_val_score(ensemble_grid, X_stack, y_stack, cv = 5, scoring = "neg_mean_squared_error") )

In [0]:
%%script false

#column Airline_Jet Airways Business, Airline_Multiple carriers Premium economy are not available in ftest
X_stack = pd.DataFrame( {'axgbpre' :base_model.predict(X_ftest) ,
                         'gbm' : gbm.predict(X_ftest),
                         'lgbm' : lgbm.predict(X_ftest),
                          'adb' : adb.predict(X_ftest)})

y_stack = y_ftest
np.sqrt( mean_squared_error(np.exp(y_ftest), np.exp(ensemble_grid.predict(X_stack))) )

## KNN

In [0]:
from sklearn.neighbors import KNeighborsRegressor

## Playground models

### Support Vector Regression - not amazing
Apply appropriate transformation

In [0]:
%%script false

from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, normalize

#scaler = StandardScaler()
#svr_model = SVR(gamma = 'scale')

svmpipe = Pipeline([ ( "std", StandardScaler() ), ( "svm", SVR(gamma = 'scale', kernel = 'rbf', C = 1.5, epsilon = 0.1, max_iter=-1) ) ])
svmpipe.fit(X_train.astype(float), y_train.astype(float))

np.sqrt( -cross_val_score(svmpipe, X_train.astype(float), y_train.astype(float), cv = 5, scoring = "neg_mean_squared_error") )
np.sqrt( mean_squared_error(np.exp(y_test), np.exp(svmpipe.predict(X_test.astype(float)))) )
pd.DataFrame( {'act' : np.exp(y_test) , 'axgbpre' :np.exp(svmpipe.predict(X_test.astype(float)))  } ).describe()

### Ridge - not a good result

In [0]:
%%script false

from sklearn.linear_model import RidgeCV

ridge_model = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1], cv = 5).fit(X_train.astype(float), y_train.astype(float))
np.sqrt( -cross_val_score(ridge_model, X_train.astype(float), y_train.astype(float), cv = 5, scoring = "neg_mean_squared_error") )
np.sqrt(mean_squared_error(npa.exp(y_test.astype(float)), np.exp(ridge_model.predict(X_test.astype(float)))))

pd.DataFrame( {'act' : np.exp(y_test) , 
               'axgbpre' :np.exp(base_model.predict(X_test)) ,
               'brfpre' : np.exp(ranger.predict(X_test)),
               'cridge' : np.exp(ridge_model.predict(X_test.astype(float))) } ).describe()

### LASSO - Not a good result

In [0]:
%%script false 

from sklearn.linear_model import LassoCV

lasso_model = LassoCV(alphas=[1e-3, 1e-2, 1e-1, 1], cv = 5).fit(X_train.astype(float), y_train.astype(float))
np.sqrt( -cross_val_score(lasso_model, X_train.astype(float), y_train.astype(float), cv = 5, scoring = "neg_mean_squared_error") )
np.sqrt(mean_squared_error(np.exp(y_test.astype(float)), np.exp(lasso_model.predict(X_test.astype(float)))))

### Neural Network - worst

In [0]:
%%script false

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense

def baseline_model():
    model = Sequential()
    model.add(Dense(100, input_dim=12, kernel_initializer='normal', activation='relu'))
    model.add(Dense(50, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
  
#model.fit(X_train, y_train)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=5, verbose = 0, validation_split = 0.20)))
pipeline = Pipeline(estimators)
#estimator = KerasRegressor(build_fn=baseline_model, epochs=10, batch_size=5, verbose=0)
# np.sqrt( -cross_val_score(pipeline, X_train.astype(float), y_train.astype(float), cv = 5, scoring = "neg_mean_squared_error") )